In [1]:
import krakenex
import time
import pandas as pd
import csv
import CryptoHelper as CH

In [2]:
df = pd.read_csv("../datasets/Kraken_ETHBTC_1h.csv", index_col=False)
df.head()

,Date,Symbol,Open,High,Low,Close,Volume ETH,Volume BTC
0,2019-10-08 10-AM,ETHBTC,0.02214,0.02215,0.02209,0.02215,213.09,4.711
1,2019-10-08 09-AM,ETHBTC,0.02207,0.02214,0.02207,0.02214,87.10,1.925
2,2019-10-08 08-AM,ETHBTC,0.02204,0.02208,0.02202,0.02207,158.68,3.501
3,2019-10-08 07-AM,ETHBTC,0.02212,0.02212,0.02202,0.02204,66.38,1.464
4,2019-10-08 06-AM,ETHBTC,0.02213,0.02232,0.02210,0.02212,1336.33,29.640


In [3]:
df.set_index("Date")

,Symbol,Open,High,Low,Close,Volume ETH,Volume BTC
Date,,,,,,,
2019-10-08 10-AM,ETHBTC,0.02214,0.02215,0.02209,0.02215,213.09,4.711
2019-10-08 09-AM,ETHBTC,0.02207,0.02214,0.02207,0.02214,87.10,1.925
2019-10-08 08-AM,ETHBTC,0.02204,0.02208,0.02202,0.02207,158.68,3.501
2019-10-08 07-AM,ETHBTC,0.02212,0.02212,0.02202,0.02204,66.38,1.464
2019-10-08 06-AM,ETHBTC,0.02213,0.02232,0.02210,0.02212,1336.33,29.640
...,...,...,...,...,...,...,...
2017-07-01 03-PM,ETHBTC,0.10830,0.11110,0.10820,0.11070,3173.37,347.320
2017-07-01 02-PM,ETHBTC,0.10940,0.10960,0.10810,0.10830,983.19,106.820
2017-07-01 01-PM,ETHBTC,0.11020,0.11070,0.10850,0.10940,1060.68,116.570


In [4]:
df = df.drop(["High", "Low", "Volume ETH", "Volume BTC"], axis=1)

In [5]:
df.reset_index(drop=True, inplace=True)

In [6]:
df.head()

,Date,Symbol,Open,Close
0,2019-10-08 10-AM,ETHBTC,0.02214,0.02215
1,2019-10-08 09-AM,ETHBTC,0.02207,0.02214
2,2019-10-08 08-AM,ETHBTC,0.02204,0.02207
3,2019-10-08 07-AM,ETHBTC,0.02212,0.02204
4,2019-10-08 06-AM,ETHBTC,0.02213,0.02212


In [7]:
k = CH.login()

In [8]:
CH.get_balance(k)

['0.0000049600', '0.1588548300']

In [51]:
def tradeBot(marginToSeek, stopLoss, startingBTC):
    currentBTC = startingBTC
    currentETH = 0
    record_transaction(currentBTC, "BTC", "start", currentBTC, currentETH)
    for i in range(0, len(df.index)):
        lastValue = startingBTC
        if (currentBTC > 0):
            #Buy Max ETH
            max_ETH = currentBTC / df.iloc[i,3] #Current Price
            currentBTC = currentBTC-currentBTC
            currentETH = currentETH + max_ETH
            record_transaction(max_ETH, "ETH","buy", currentBTC, currentETH)
        else:
            currentBTCVal = currentETH * df.iloc[i,3]
            targetBTCVal = lastValue*marginToSeek
            if(currentBTCVal > targetBTCVal):
                max_BTC = currentETH * df.iloc[i,3]
                currentBTC = currentBTC + max_BTC
                record_transaction(currentETH, "ETH", "sell", currentBTC, currentETH - currentETH)
                currentETH = currentETH - currentETH
                lastValue = targetBTCVal
        if (i== len(df.index) - 1):
            #Sell all ETH
            max_BTC = currentETH * df.iloc[i,3]
            currentBTC = currentBTC + max_BTC
            record_transaction(currentETH, "ETH", "sell", currentBTC, currentETH - currentETH)
            currentETH = currentETH - currentETH
            lastValue = targetBTCVal
            return [currentBTC , currentETH]

In [10]:
def record_transaction(amount, coin, action, btcBal, ethBal):
    with open('transactions.csv', mode='a') as transactionFile:
        transactionFile = csv.writer(transactionFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        transactionFile.writerow([amount, coin, action, btcBal, ethBal])


In [52]:
final = tradeBot(1.5,0,0.5)
print(final)

[1.2911742930210148, 0.0]
